<a href="https://colab.research.google.com/github/vunameaut/Predict-age-and-gender/blob/main/Predict_age_and_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tải và giải nén dữ liệu từ Google Drive
import os
import zipfile
import gdown

# Đường dẫn tải dữ liệu từ Google Drive
file_id = "11Mf5iOX2Zr6h30zkepljem8qzVyn40OZ"  # ID tệp Google Drive
DATA_PATH = "/content/UTKFace.zip"  # Lưu tệp zip
BASE_DIR = "/content"  # Thư mục giải nén dự kiến

def download_and_extract_data():
    print("Đang tải dữ liệu từ Google Drive...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", DATA_PATH, quiet=False)

    # Kiểm tra tệp tải xuống
    if os.path.exists(DATA_PATH) and os.path.getsize(DATA_PATH) > 0:
        print(f"Tệp đã tải xuống thành công: {DATA_PATH}")
        print(f"Kích thước: {os.path.getsize(DATA_PATH)} bytes")
    else:
        raise ValueError("Tệp tải xuống không tồn tại hoặc rỗng. Kiểm tra lại liên kết Google Drive.")

    # Giải nén tệp ZIP nếu cần
    if DATA_PATH.endswith(".zip"):
        print("Giải nén tệp ZIP...")
        try:
            with zipfile.ZipFile(DATA_PATH, 'r') as zip_ref:
                zip_ref.extractall('/content/')
                print("Giải nén thành công.")
        except zipfile.BadZipFile:
            raise ValueError("Tệp không phải định dạng ZIP hợp lệ. Hãy kiểm tra tệp nguồn!")
        except Exception as e:
            raise ValueError(f"Lỗi khi giải nén tệp ZIP: {str(e)}")
    else:
        raise ValueError(f"Tệp không phải định dạng ZIP: {DATA_PATH}")

    # Kiểm tra thư mục sau giải nén
    extracted_dirs = [d for d in os.listdir('/content/') if os.path.isdir(os.path.join('/content/', d))]
    if len(extracted_dirs) == 0:
        raise ValueError("Không tìm thấy thư mục dữ liệu sau khi giải nén. Kiểm tra lại tệp!")
    BASE_DIR = os.path.join('/content/', extracted_dirs[0])
    print(f"Thư mục dữ liệu được đặt là: {BASE_DIR}")

    return BASE_DIR

# Gọi hàm để tải và giải nén dữ liệu
BASE_DIR = download_and_extract_data()


Đang tải dữ liệu từ Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=11Mf5iOX2Zr6h30zkepljem8qzVyn40OZ
From (redirected): https://drive.google.com/uc?id=11Mf5iOX2Zr6h30zkepljem8qzVyn40OZ&confirm=t&uuid=014c6f97-ef4a-4e9b-94fe-42653f0ef4e1
To: /content/UTKFace.zip
100%|██████████| 121M/121M [00:01<00:00, 98.8MB/s]


Tệp đã tải xuống thành công: /content/UTKFace.zip
Kích thước: 121296961 bytes
Giải nén tệp ZIP...
Giải nén thành công.
Thư mục dữ liệu được đặt là: /content/.config


In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, BatchNormalization
import tensorflow as tf
from PIL import Image
from tqdm.auto import tqdm
from sklearn.utils import class_weight

# Đảm bảo sử dụng GPU nếu có
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Sử dụng GPU để train.")
    except RuntimeError as e:
        print(e)
else:
    print("Không tìm thấy GPU, đang sử dụng CPU.")

# Đường dẫn tới thư mục chứa dữ liệu
BASE_DIR = '/content/UTKFace'

# Đọc và chuẩn bị dữ liệu
def load_data(BASE_DIR):
    print("Đang load dữ liệu...")
    image_paths = []
    age_labels = []
    gender_labels = []

    for filename in tqdm(os.listdir(BASE_DIR)):
        temp = filename.split('_')
        if len(temp) < 2:
            continue
        try:
            age = int(temp[0])
            gender = int(temp[1])
        except ValueError:
            continue

        image_path = os.path.join(BASE_DIR, filename)
        image_paths.append(image_path)
        age_labels.append(age)
        gender_labels.append(gender)

    df = pd.DataFrame()
    df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels

    # Nhóm tuổi theo khoảng
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    df['age_bins'] = pd.cut(df['age'], bins=bins, labels=False)
    return df

# Tính năng trích xuất ảnh
def extract_features(images, target_size=(128, 128)):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = img.resize(target_size, Image.Resampling.LANCZOS)
        img = np.array(img)
        features.append(img)

    features = np.array(features)
    features = features.reshape(len(features), *target_size, 1)
    return features

def build_and_train_model(df):
    # Tiền xử lý dữ liệu
    X = extract_features(df['image'])
    X = X / 255.0
    y_gender = np.array(df['gender'])
    y_age = np.array(df['age_bins'])

    # Tính class weights cho giới tính
    class_weights = class_weight.compute_class_weight(
        'balanced', classes=np.unique(y_gender), y=y_gender
    )
    class_weights_dict = dict(enumerate(class_weights))

    # Xây dựng mô hình
    input_shape = (128, 128, 1)
    inputs = Input((input_shape))

    # Các lớp CNN
    conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
    batchnorm_1 = BatchNormalization()(conv_1)
    maxp_1 = MaxPooling2D(pool_size=(2, 2))(batchnorm_1)

    conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(maxp_1)
    batchnorm_2 = BatchNormalization()(conv_2)
    maxp_2 = MaxPooling2D(pool_size=(2, 2))(batchnorm_2)

    conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu')(maxp_2)
    batchnorm_3 = BatchNormalization()(conv_3)
    maxp_3 = MaxPooling2D(pool_size=(2, 2))(batchnorm_3)

    conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu')(maxp_3)
    batchnorm_4 = BatchNormalization()(conv_4)
    maxp_4 = MaxPooling2D(pool_size=(2, 2))(batchnorm_4)

    flatten = Flatten()(maxp_4)

    # Nhánh 1: Dự đoán giới tính
    dense_1 = Dense(256, activation='relu')(flatten)
    dropout_1 = Dropout(0.3)(dense_1)
    output_1 = Dense(1, activation='sigmoid', name='gender_out')(dropout_1)

    # Nhánh 2: Dự đoán tuổi
    dense_2 = Dense(256, activation='relu')(flatten)
    dropout_2 = Dropout(0.3)(dense_2)
    output_2 = Dense(len(np.unique(y_age)), activation='softmax', name='age_out')(dropout_2)

    model = Model(inputs=[inputs], outputs=[output_1, output_2])

    # Huấn luyện nhánh dự đoán giới tính
    print("Huấn luyện nhánh dự đoán giới tính...")
    gender_model = Model(inputs=model.input, outputs=model.get_layer('gender_out').output)
    gender_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',  # Loss cho giới tính
        metrics=['accuracy']  # Accuracy cho giới tính
    )
    gender_model.fit(
        x=X,
        y=y_gender,
        batch_size=16,
        epochs=30,
        validation_split=0.2,
        class_weight=class_weights_dict  # Áp dụng class_weight
    )

    # Huấn luyện nhánh dự đoán tuổi
    print("Huấn luyện nhánh dự đoán tuổi...")
    age_model = Model(inputs=model.input, outputs=model.get_layer('age_out').output)
    age_model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',  # Loss cho tuổi
        metrics=['accuracy']  # Accuracy cho tuổi
    )
    age_model.fit(
        x=X,
        y=y_age,
        batch_size=16,
        epochs=30,
        validation_split=0.2
    )

    return model



# Lưu mô hình
def save_models(model):
    output_dir = '/content'
    os.makedirs(output_dir, exist_ok=True)

    gender_model = Model(inputs=model.input, outputs=model.get_layer('gender_out').output)
    gender_model.save(os.path.join(output_dir, 'gender_model.h5'))

    age_model = Model(inputs=model.input, outputs=model.get_layer('age_out').output)
    age_model.save(os.path.join(output_dir, 'age_model.h5'))

    print("Mô hình đã được lưu thành công!")

# Gọi các hàm
df = load_data(BASE_DIR)
model = build_and_train_model(df)
save_models(model)


Sử dụng GPU để train.
Đang load dữ liệu...


  0%|          | 0/23708 [00:00<?, ?it/s]

  0%|          | 0/23708 [00:00<?, ?it/s]

Huấn luyện nhánh dự đoán giới tính...
Epoch 1/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 30s 18ms/step - accuracy: 0.7553 - loss: 0.6983 - val_accuracy: 0.7938 - val_loss: 0.4406
Epoch 2/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 29s 12ms/step - accuracy: 0.8667 - loss: 0.2951 - val_accuracy: 0.8583 - val_loss: 0.3240
Epoch 3/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.8886 - loss: 0.2616 - val_accuracy: 0.8602 - val_loss: 0.3110
Epoch 4/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9005 - loss: 0.2360 - val_accuracy: 0.8224 - val_loss: 0.4528
Epoch 5/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.9006 - loss: 0.2385 - val_accuracy: 0.8872 - val_loss: 0.2603
Epoch 6/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.9187 - loss: 0.1957 - val_accuracy: 0.8655 - val_loss: 0.3138
Epoch 7/30
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.9289 - loss: 0.1711 - val_accuracy: 0.5333 - val_loss: 9.3335
Epoch 8/30
1186/1186 ━━━━━━━━━━━━━━━━━

Mô hình đã được lưu thành công!
